In [1]:
import glob
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.common.features.feature_metadata import FeatureMetadata

In [2]:
training_files = 3
data_files = glob.glob('play_data/*.tsv')
dfs = [pd.read_csv(file, sep='\t') for file in data_files[:training_files]]

train_data = TabularDataset(pd.concat(dfs, ignore_index=True))
train_data['play_pos'] = train_data['trick_num'] * 4 + train_data['play_seq']
train_data['play_pos'] = train_data['play_pos'].astype('category')

In [3]:
train_data.head()

,run_id,key,trick_num,play_seq,pos,partner_winning,lead_trumped,tricks_won,trumps_seen,turn_seen,...,throw_off_long,throw_off_low,throw_off_next,tricks_min,tricks_max,tricks_avg,points_min,points_max,points_avg,play_pos
0,953904,10♣ 9♥ A♣ J♥ K♣,5,1,0,0,0,1,6,3,...,0,0,0,1,1,1.0,-2,-2,-2.0,21
1,953904,10♣ 9♥ J♥ K♣ A♣,5,1,0,0,0,0,6,3,...,0,0,0,1,1,1.0,-2,-2,-2.0,21
2,953904,9♥ 10♣ J♥ A♣ K♣,5,0,0,0,0,1,5,3,...,0,0,0,2,2,2.0,-2,-2,-2.0,20
3,953904,10♣ J♥ 9♥ A♣ K♣,5,1,0,0,0,0,6,3,...,0,0,0,1,1,1.0,-2,-2,-2.0,21
4,953904,10♣ J♥ K♣ 9♥ A♣,5,1,0,0,0,1,6,3,...,0,0,0,1,1,1.0,-2,-2,-2.0,21


In [4]:
label = 'points_max'
train_data[label].describe()

count    587716.000000
mean         -0.083217
std           1.644207
min          -4.000000
25%          -1.000000
50%          -1.000000
75%           1.000000
max           4.000000
Name: points_max, dtype: float64

In [5]:
exclude = ['run_id', 'key', 'trick_num', 'play_seq']
outputs = ['tricks_min', 'tricks_max', 'tricks_avg', 'points_min', 'points_max', 'points_avg']
outputs.remove(label)

quantile_levels = [0.25, 0.5, 0.75]
predictor = TabularPredictor(problem_type='quantile', quantile_levels=quantile_levels, label=label)
predictor.fit(train_data.drop(columns=(exclude + outputs)), presets='best_quality')

No path specified. Models will be saved in: "AutogluonModels/ag-20240906_044219"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.9
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #202405300957~1722440358~24.04~aa0a43f~dev-Ubuntu SMP PREEMPT_DY
CPU Count:          8
Memory Avail:       11.39 GB / 15.34 GB (74.3%)
Disk Space Avail:   117.92 GB / 452.95 GB (26.0%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of Auto

In [6]:
test_data = TabularDataset(data_files[training_files])[:50000]
test_data['play_pos'] = test_data['trick_num'] * 4 + test_data['play_seq']

y_pred = predictor.predict(test_data.drop(columns=([label] + exclude + outputs)))
y_pred.head()

Loaded data from: play_data/play_model_1-20240903_111103.tsv | Columns = 56 / 56 | Rows = 192843 -> 192843


,0.25,0.50,0.75
0,0.876788,0.999164,1.001939
1,-2.024170,-2.017102,-1.811369
2,-2.034576,-1.679188,-1.091842
3,0.997099,1.066863,1.728191
4,0.994963,1.002625,0.997357


In [8]:
predictor.evaluate(test_data.drop(columns=(exclude + outputs)), silent=True)

{'pinball_loss': -0.18117485731512695}

In [9]:
predictor.leaderboard(test_data.drop(columns=(exclude + outputs)))

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT_BAG_L2,-0.181175,-0.143050,pinball_loss,249.929711,5906.872751,1865.535011,44.339312,954.181970,467.214406,2,True,3
1,WeightedEnsemble_L3,-0.181175,-0.143050,pinball_loss,249.935913,5906.928915,1868.461773,0.006203,0.056164,2.926762,3,True,4
2,LightGBMXT_BAG_L1,-0.187892,-0.161225,pinball_loss,205.590399,4952.690782,1398.320606,205.590399,4952.690782,1398.320606,1,True,1
3,WeightedEnsemble_L2,-0.187892,-0.161225,pinball_loss,205.593338,4952.761336,1398.404459,0.002940,0.070554,0.083854,2,True,2
